## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [0]:
! pip install haversine

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [0]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [0]:
# answer goes here
df = pd.read_csv('https://raw.githubusercontent.com/qboy21/Thinkful/master/starbucks_locations.csv')


Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [0]:
# answer goes here
az = df[(df['State/Province']=='AZ')&(df['Country']=='US')]
az.info()
az.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 12153 to 12640
Data columns (total 13 columns):
Brand             488 non-null object
Store Number      488 non-null object
Store Name        488 non-null object
Ownership Type    488 non-null object
Street Address    488 non-null object
City              488 non-null object
State/Province    488 non-null object
Country           488 non-null object
Postcode          488 non-null object
Phone Number      474 non-null object
Timezone          488 non-null object
Longitude         488 non-null float64
Latitude          488 non-null float64
dtypes: float64(2), object(11)
memory usage: 53.4+ KB


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
12153,Starbucks,21989-106025,Fry's #78 Anthem,Licensed,39508 N. Daisy Mountain Drive,Anthem,AZ,US,850866056,623-551-7220,GMT+000000 America/Phoenix,-112.13,33.84
12154,Starbucks,6581-67540,"Anthem & I-17, Anthem",Company Owned,3640 W. Anthem Way,Anthem,AZ,US,850860416,623-551-5407,GMT+000000 America/Phoenix,-112.14,33.87
12155,Starbucks,14177-106635,Daisy Mountain & Gavilan Peak,Company Owned,"39504 N. Daisy Mountain Drive, #118",Anthem,AZ,US,850866065,623-551-0977,GMT+000000 America/Phoenix,-112.13,33.84
12156,Starbucks,3765-148824,"Fry's-Apache Junction, AZ #65",Licensed,185 W Apache Tr,Apache Junction,AZ,US,851203433,480-288-2140,GMT+000000 America/Phoenix,-111.55,33.41
12157,Starbucks,9862-97880,Apache Trail & Delaware,Company Owned,2580 W. Apache Trail,Apache Junction,AZ,US,851205203,480-982-2346,GMT+000000 America/Phoenix,-111.57,33.42


The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
-121.64, 39.14  
-116.40, 34.13  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
-121.64, 39.14, (-121.64, 39.14)  
-116.40, 34.13, (-116.40, 34.13)  
...

In [0]:
# answer goes here
az['Coordinates'] = az[['Latitude','Longitude']].apply(tuple, axis=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [0]:
# answer goes here
az_dist = geo_sim_matrix(az)



In [0]:
az_dist.head()

index,12153,12154,12155,12156,12157,12158,12159,12160,12161,12162,12163,12164,12165,12166,12167,12168,12169,12170,12171,12172,12173,12174,12175,12176,12177,12178,12179,12180,12181,12182,12183,12184,12185,12186,12187,12188,12189,12190,12191,12192,...,12601,12602,12603,12604,12605,12606,12607,12608,12609,12610,12611,12612,12613,12614,12615,12616,12617,12618,12619,12620,12621,12622,12623,12624,12625,12626,12627,12628,12629,12630,12631,12632,12633,12634,12635,12636,12637,12638,12639,12640
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12153,0.000000,2.150754,0.000000,44.678482,43.358982,28.900461,27.636618,30.161417,25.873920,166.846298,212.029373,37.608036,165.678833,163.199478,53.087055,52.745056,12.131993,68.735863,70.423187,66.399590,9.042999,9.591160,10.163091,46.865055,38.493053,37.251549,39.590267,45.223156,44.053120,38.939548,44.924011,38.057266,46.473602,48.639935,40.115852,39.990067,39.781456,39.590267,38.095039,41.507336,...,125.826134,127.783623,134.803864,133.640305,128.831192,121.034248,118.658470,136.779526,117.164604,130.557983,136.779526,129.065613,136.872131,143.180923,130.580551,129.073608,130.255722,133.622574,134.079727,130.557983,131.247055,125.826134,135.157013,127.783623,132.853470,126.695541,123.489639,131.451965,133.554916,37.627735,171.764572,97.481537,163.483505,159.965454,154.923264,166.188400,165.016815,162.977600,156.034241,166.188400
12154,2.150754,0.000000,2.150754,46.496811,45.184311,30.573988,29.450094,31.934774,27.635677,168.809113,214.018417,38.829971,164.076141,161.633804,51.255665,50.942207,13.088914,70.882156,72.568130,68.546707,10.378566,10.889643,11.954039,48.992966,40.633984,39.375797,41.735718,47.340950,46.189075,41.084099,47.067394,40.207977,48.595898,50.738586,42.228897,42.130753,41.914268,41.735718,40.240814,43.636520,...,127.883278,129.818466,136.895065,135.734100,130.920151,123.103867,120.734337,138.809097,119.167747,132.622391,138.809097,131.141251,138.894608,145.224396,132.638641,131.137039,132.322418,135.669876,136.110825,132.622391,133.293701,127.883278,137.198914,129.818466,134.887634,128.758606,125.539291,133.515717,135.638367,36.654388,173.515701,95.391350,163.967834,160.549500,155.517319,166.673233,165.520233,163.465622,156.608109,166.673233
12155,0.000000,2.150754,0.000000,44.678482,43.358982,28.900461,27.636618,30.161417,25.873920,166.846298,212.029373,37.608036,165.678833,163.199478,53.087055,52.745056,12.131993,68.735863,70.423187,66.399590,9.042999,9.591160,10.163091,46.865055,38.493053,37.251549,39.590267,45.223156,44.053120,38.939548,44.924011,38.057266,46.473602,48.639935,40.115852,39.990067,39.781456,39.590267,38.095039,41.507336,...,125.826134,127.783623,134.803864,133.640305,128.831192,121.034248,118.658470,136.779526,117.164604,130.557983,136.779526,129.065613,136.872131,143.180923,130.580551,129.073608,130.255722,133.622574,134.079727,130.557983,131.247055,125.826134,135.157013,127.783623,132.853470,126.695541,123.489639,131.451965,133.554916,37.627735,171.764572,97.481537,163.483505,159.965454,154.923264,166.188400,165.016815,162.977600,156.034241,166.188400
12156,44.678482,46.496811,44.678482,0.000000,1.344559,45.681511,42.232697,43.850285,43.016975,122.872505,168.303131,58.262070,209.964279,207.342773,83.019989,82.092171,35.436325,36.911789,37.917126,35.318733,36.584270,36.197529,34.544823,19.511490,20.581795,18.359835,21.581848,17.724865,20.062035,21.347229,21.794785,24.095461,18.619274,16.743711,16.775627,20.505571,19.177683,21.581848,21.684774,18.678574,...,83.756287,85.109413,93.838684,92.802551,87.848778,79.419212,77.301636,93.925827,73.913818,88.661247,93.925827,87.561249,93.870087,100.606514,88.491486,87.160889,88.434715,91.210426,91.276009,88.661247,88.836555,83.756287,92.597099,85.109413,90.126633,84.796776,81.230087,89.525909,92.288979,79.772888,127.152565,132.271378,183.862366,178.639160,173.569824,186.484085,185.011520,183.304382,175.016769,186.484085
12157,43.358982,45.184311,43.358982,1.344559,0.000000,44.

In [0]:
az.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,"(24.48, 54.38)"
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,"(24.51, 54.54)"
5,Starbucks,17688-182164,"Dalma Mall, Ground Floor",Licensed,"Dalma Mall, Mussafah",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.49,24.40,"(24.4, 54.49)"
6,Starbucks,18182-182165,"Dalma Mall, Level 1",Licensed,"Dalma Mall, Mussafah",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.49,24.40,"(24.4, 54.49)"
7,Starbucks,23359-229184,Debenhams Yas Mall,Licensed,Yas Island,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.61,24.46,"(24.46, 54.61)"


For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [0]:
# answer goes here
az2 = pd.DataFrame()

az2['Location'] = az['Store Number']

az2.head()

,Location
12153,21989-106025
12154,6581-67540
12155,14177-106635
12156,3765-148824
12157,9862-97880


In [0]:
for ind in (az2.index):
    check = az_dist[ind].where(az_dist[ind].index != ind)
    i = check.idxmin()
    #az2['Nearest Neighbor'] = az['Store Number'][i]
    az2['Distance'] = az_dist[ind][i]
    

In [59]:
# nn = nearest neighbor
nn ={}
for i in az_dist.index:
    distances = az_dist[i][az_dist.index !=i]
    nn[i,distances.idxmin()] = min(distances)

nn = pd.DataFrame(list(nn.items()))
nn['Location'] = nn[0].apply(lambda x: x[0])
nn['Nearest Neighbor'] = nn[0].apply(lambda x: x[1])
nn['Distance (Miles)'] = nn[1]
nn.drop(columns=[0,1], inplace = True)
nn.head()



,Location,Nearest Neighbor,Distance (Miles)
0,12153,12155,0.000000
1,12154,12153,2.150754
2,12155,12153,0.000000
3,12156,12157,1.344559
4,12157,12156,1.344559


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [0]:
# answer goes here



